# Исследование уровня потребительской лояльности

- Цель исследования - определение текущего уровня потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. Для наглядности результата будет построен дашборд.
- Методика расчёта: клиентам задавали вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». Оценки делят на 3 группы: 9-10 баллов — «сторонники» (англ. promoters), 7-8 баллов — «нейтралы» (англ. passives), 0-6 баллов — «критики» (англ. detractors). Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
- Заказчик — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.
- Данные предоставлены заказчиком (по результатам опроса данные были внесены в БД SQLite).

In [2]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

Делаем SQL-запрос для выгрузки данных, затем загружаем CSV-файл на локальный компьютер.

In [4]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'новый'
           ELSE 'старый'
       END AS is_new,
       u.age,
       CASE
           WHEN u.gender_segment = 0 THEN 'мужчина'
           WHEN u.gender_segment = 1 THEN 'женщина'
           ELSE 'не определено'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       ag.title AS age_segment,
       ts.title AS trafic_segment,
       ls.title AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score <= 6 THEN 'критик'
           WHEN u.nps_score >= 9 THEN 'сторонник'
           ELSE 'нейтрал'
           END AS nps_group
FROM user AS u
INNER JOIN location AS l ON u.location_id=l.location_id
INNER JOIN age_segment AS ag ON u.age_gr_id=ag.age_gr_id
INNER JOIN traffic_segment AS ts ON u.tr_gr_id=ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id;
"""

In [5]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,trafic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонник
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонник
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонник


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/views/project_nps/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link

### Ссылка на файл с презентацией:

https://docs.google.com/presentation/d/1a_Or8F27ValpD7iFgJQkGiRPNt9Hq6lRZBIrbUfCNkU/edit?usp=sharing